# CSM voice synthesis

In [1]:
import os
import math

import librosa
import librosa.display

import numpy as np

In [2]:
# Sine wave generator
class sine_gen:
    def __init__(self):
        self.phase = 0
        self.set_frequency(1000)
        self.set_sample_rate(44100)

    def generate_sine_wave(self):
        self.wave = [ math.sin((step * (2 * math.pi)) / self.sample_rate) for step in range(self.sample_rate) ]

    def read(self):
        res = self.wave[int(self.phase)]
        self.phase += self.frequency
        if self.phase >= self.sample_rate:
            self.phase -= self.sample_rate
        return res

    def set_frequency(self, frequency):
        self.frequency = frequency
        #self.phase = 0

    def set_sample_rate(self, sample_rate):
        self.sample_rate = sample_rate
        self.generate_sine_wave()

## Read CSM file

Time resolution is 10ms. Each line contains freq and amp data for 10ms period.    

**Note**: CSM file line format  
```
freq1,amp1,freq2,amp2,freq3,amp3,freq4,amp4
```

In [3]:
input_file = 'apollo11_launch.csm'
with open(input_file, 'r') as f:
    lines = [[float(dt) for dt in (l.rstrip('\n')).split(',')] for l in f.readlines()]

# CSM data = [ [[f1,a1],[f2,a2],[f3,a3][f4,a4]], [[f1,a1],[f2,a2]].... ]
csm_data = []
for line in lines:
    item = []
    for idx in range(4):
        item.append([line[idx*2], line[idx*2+1]])
    csm_data.append(item)

print(len(csm_data) * 0.01, 'sec')

25.26 sec


## CSM voice synthesis

In [4]:
ops = [ sine_gen() for i in range(4) ]  # CSM operators = sine wave generators

output = []
for csm in csm_data:
    for i in range(4):
        freq = csm[i][0]
        ops[i].set_frequency(freq)
    for t in range(int(44100 * 10e-3)):  # Generate CSM wave for 10ms.
        val = 0
        for i in range(4):
            amp = csm[i][1]
            val += ops[i].read() * amp   # CSM
        output.append(val)

output = np.array(output, dtype=np.float32)

# normalize
output = (output-output.min()) / (output.max()-output.min())
output = (output-0.5) * 32767
output = output.astype(np.int16)
print(output.min(), output.max())

-16383 16383


## Playback generated audio data

In [5]:
import IPython
IPython.display.Audio(output, rate=44100, autoplay=True)

## Save audio file (input_file_out.wav)

In [6]:
import wave
base, _ = os.path.splitext(input_file)
output_file = base+'_out.wav'
fout = wave.open(output_file, 'wb')
fout.setparams((1, 2, 44100, len(output), 'NONE', 'notcompressed'))
fout.writeframes(bytearray(output))
fout.close()
print(output_file, 'is saved.')

apollo11_launch_out.wav is saved.
